<a href="https://colab.research.google.com/github/notimeforcaution/airelius/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### a(i)relius

In [5]:
BASE_PATH = "/content/drive/MyDrive/colab_notebooks/airelius/"

In [63]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [26]:
# Creating dataset.

from datasets import Dataset

with open(BASE_PATH + "data/meditations.txt", "r") as f:
  text = f.read()

dataset = Dataset.from_dict({"text": text.splitlines()})

<class 'datasets.arrow_dataset.Dataset'>


In [38]:
# Tokenizing.

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

# Artificially adding a padding token.
def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Adding labels.
def add_labels(examples):
  examples["labels"] = examples["input_ids"].copy()
  return examples

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
tokenized_datasets = tokenized_datasets.map(add_labels, batched=False)


Map (num_proc=4):   0%|          | 0/35 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [39]:
# Set-up training args.

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
)

In [40]:
# Set-up trainer.

from transformers import Trainer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    # TODO(ntfc): Try using split dataset for evaluation.
    eval_dataset=tokenized_datasets,
)

In [41]:
trainer.train()  # TODO(ntfc): Tune the model better, reduce loss.

Step,Training Loss


TrainOutput(global_step=54, training_loss=0.8239091943811487, metrics={'train_runtime': 1211.7514, 'train_samples_per_second': 0.087, 'train_steps_per_second': 0.045, 'total_flos': 27435663360000.0, 'train_loss': 0.8239091943811487, 'epoch': 3.0})

In [44]:
trainer.save_model("./fine-tuned-model")
model.save_pretrained("./fine-tuned-model")

In [64]:
# Reflecting on results

input_prompt = "Reflect on the nature of life: "
input_ids = tokenizer.encode(input_prompt, return_tensors="pt")

# TODO(ntfc): Make reasonable decision on generation params.
output = model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.4, num_beams=5)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
